### Look at: what is the relationship between residual R^2 and partial R^2?
Would expect them to be equal, if not very correlated

In [3]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import pandas as pd
import utils.behavioral_utils as behavioral_utils
import utils.information_utils as information_utils
import utils.visualization_utils as visualization_utils
import utils.glm_utils as glm_utils
from matplotlib import pyplot as plt
import utils.spike_utils as spike_utils
from constants.glm_constants import *
from constants.behavioral_constants import *
from spike_tools import (
    general as spike_general,
    analysis as spike_analysis,
)

In [4]:
OUTPUT_DIR = "/data/patrick_res/glm_2"
SESSIONS_PATH = "/data/patrick_res/sessions/valid_sessions_rpe.pickle"
sessions = pd.read_pickle(SESSIONS_PATH)

In [5]:
positions = spike_utils.get_unit_positions(sessions)

### Load residual models

In [6]:
def load_glm(session, res_name):
    res = pd.read_pickle(os.path.join(OUTPUT_DIR, res_name))
    res["TimeIdxs"] = (res["TimeBins"] * 20).astype(int)
    res["session"] = session
    res["PseudoUnitID"] = int(session) * 100 + res.UnitID
    return res


def get_residual_glm_res(session):
    res_name = f"{session}_glm_RPEGroup_residual_fr_{MODE}_{INTERVAL_SIZE}_{MODEL}_values.pickle"
    res = load_glm(session, res_name)
    return res

In [7]:
residual_glm_res = pd.concat(sessions.apply(lambda x: get_residual_glm_res(x.session_name), axis=1).values)

In [14]:
merged = pd.merge(residual_glm_res, positions, on="PseudoUnitID")
merged[merged.score > 0.1].groupby("structure_level2").PseudoUnitID.nunique()

structure_level2
basal_ganglia (BG)                     4
lateral_prefrontal_cortex (lat_PFC)    4
Name: PseudoUnitID, dtype: int64

### Load and calculate partial r^2

In [13]:
def get_additional_score(session):
    with_vals_path = f"{session}_glm_RPEGroup_normal_fr_{MODE}_{INTERVAL_SIZE}_{MODEL}_values.pickle"
    with_vals_res = load_glm(session, with_vals_path)
    with_vals_res = with_vals_res.rename(columns={"score": "with_vals_score"})


    without_vals_path = f"{session}_glm_{MODE}_{INTERVAL_SIZE}_{MODEL}_feature_rpe_interaction.pickle"
    without_vals_res = load_glm(session, without_vals_path)
    print(without_vals_res.columns)
    without_vals_res = without_vals_res.rename(columns={"score": "without_vals_score"})

    merged = pd.merge(with_vals_res, without_vals_res, on=["UnitID", "TimeIdxs"])
    print(merged.columns)
    merged["additional_score"] = 1 - (1 - merged["with_vals_score"]) / (1 - merged["without_vals_score"])
    return merged

In [14]:
partial_reses = pd.concat(sessions.apply(lambda x: get_additional_score(x.session_name), axis=1).values)

Index(['UnitID', 'TimeBins', 0, 'TimeIdxs', 'session', 'PseudoUnitID'], dtype='object')
Index([                'UnitID',             'TimeBins_x',
              'with_vals_score',          'less neg_coef',
                'more pos_coef',          'more neg_coef',
                'less pos_coef',              'None_coef',
                  'YELLOW_coef',              'CYAN_coef',
                 'MAGENTA_coef',             'GREEN_coef',
                  'CIRCLE_coef',              'STAR_coef',
                  'SQUARE_coef',          'TRIANGLE_coef',
                'POLKADOT_coef',             'SWIRL_coef',
                  'ESCHER_coef',            'RIPPLE_coef',
         'YELLOW_less neg_coef',     'CYAN_less neg_coef',
        'MAGENTA_less neg_coef',  'MAGENTA_more pos_coef',
           'CYAN_more pos_coef',     'CYAN_more neg_coef',
          'GREEN_less neg_coef',   'YELLOW_more pos_coef',
          'GREEN_more pos_coef',   'YELLOW_less pos_coef',
        'MAGENTA_less pos_c

KeyError: 'without_vals_score'